# A simple first introduction to reinforcement learning 
<br>

### Briefly Reinforcement Learning is : 
<br>
In an environnement E, an agent in a state S takes an action to go to another state S' and receive a reward R.
The choice of the action to take is defined by a policy P which that defines for each state what is the best action.

<br>


To illustrate this simply, we will use a simple use case : a robot that has to navigate to reach a goal avoiding fire that can damage him.
    

<img src="img/GridRL.png" width="60%">

<br>
Although this environnement doesn't have so many states, 9 only, the number of possible actions depending on the
current state is actually high. We have to define by hand the reward for taking an action in each state, and whenever it is not possible, it is defined by Nan.

<img src="img/ArrayPossibleActionsRL.png" width="60%">

In [15]:
import numpy as np 

In [16]:
# We replace Nan by -100 like if it is a big punishment
R = [[0,-100,-100,0],[0,-100,0,0],[-1,-100,0,-100],[1,0,0,-100],[0,0,0,-1],[-1,0,-100,0],[-100,0,-100,0],[-100,0,-1,1],[-100,-1,0,-100]]

### Navigating through the environement

Now we need to compute the policy P that doesn't only take into account the current reward but future rewards as well. We will use a well-known method : Q-learning. 
<br>

The Q-learning method gives us a Q(s,a) function representing the value (function of reward) of taking an action a in state s assuming that the agent follows the policy. The agent searching for the optimal policy can then be described using the Bellman optimality equation as : 
<br>

$$ Q^{*}(s_{t}, a_{t}) = r(s_{t+1}, a_{t+1}) + \gamma \times \max_{a_{t+1}}Q^{*}(s_{t+1}, a_{t+1}) $$ 
<br>
Which means the Q-value at time t equals the sum of the immediate reward and the estimate of optimal future value.
The parameter $ \gamma $ is called the discount factor, it gives more importance to close rewards than future reward.

<br>
A common way of choosing which action to take is the $ \epsilon $-greedy strategy. Instead of always taking the best action, we can force the agent to fully explore the environement by taking random actions with probability $ \epsilon $, and best actions with probability $ 1 - \epsilon $

In [17]:
# Back to programming we can initialize the Q look-up table
Q = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]

# We also define an array that maps action to states to know in which state when taking an action 
# 0 means that is action is not possible because the next state doesn't exist
ActionToState = [[6,0,0,2], [5,0,1,3],[4,0,2,0],[9,3,5,0],[8,2,6,4],[7,1,0,5],[0,6,0,8],[0,5,7,9],[0,4,8,0]]

In [18]:
# Learning parameter 
gamma = 0.7

# n_episode correspond to number of times the robot will try to catch the diamond
n_episode = 10000  
next_state = 0

for i in range(n_episode) : 

    # Initilialize current state and nb of try at each start of an episode
    current_state = 0
    nb_try = 0

    
    # An episode stops if the robots reached the diamond or if he's been wandering too much
    while (state!=8 or nb_try<50) :

        # In this loop the robot takes an action, it the action is not possible then he tries again.
        # Also the action is taken following the epsilon greedy strategy to fully explore the environement 
        while True :
            
            epsilon = np.random.random()
            
            if (epsilon>0.3) : 
                
                # Greedy choice of action => Good for exploration
                action = np.argmax(Q[current_state][:])  
            
            else :  
            
                # Random choice of action
                action = int(round(np.random.random()*3)) 
                
            # We compute the next_state from the array defined above
            next_state = ActionToState[current_state][action] -1
            
            # If the action is not feasible then the robot has to take antoher action
            if (next_state!=-1) : break

        # Q value is updated with the sum of : 
        #     _ current reward defined by the R array according to current state and action
        #     _ the discounted future Q value according to next state
        Q[current_state][action] = R[current_state][action] + gamma*max(Q[next_state][:])

        
        # Update current state and nb of try for next iteration
        current_state = next_state
        nb_try += 1

In [19]:
# We can print what is the best action depending on each current state

# First let's define a dict mapping an index to an action
Index2Action = {0 : "Up", 1 : "Down", 2 : "Left", 3 : "Right"}

# Then we can just loop through each state and print the argmax of the Q value along each line 
count = 1
for i in range(9) :
    BestAction = Index2Action[np.argmax(Q[i][:])]
    print(str(count) + " : " + BestAction)
    count+=1

1 : Up
2 : Up
3 : Left
4 : Up
5 : Up
6 : Right
7 : Right
8 : Right
9 : Left
